# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
DNS and physics-informed surrogate models of surfactant-laden dispersed flows	Oral Presentation	talk-1		2023-11-20	Washington D.C., USA		76th Annual Meeting of the American Physical Society (APS), Division of Fluid Dynamics (DFD)
Direct Numerical Simulation of an Alternative SMX Element Orientation for Laminar Liquid-Liquid Mixing 	Oral Presentation	talk-2		2023-11-07	Orlando, USA		2023 AIChE Annual Meeting
Direct numerical simulations of the dispersion dynamics of complex flows in static mixers with surfactants 	Oral Presentation	talk-3		2023-04-06	Kobe, Japan		The 11th International Conference on Multiphase Flow
Direct numerical simulations of the dispersion dynamics of liquid-liquid surfactant-laden flows in static mixers 	Oral Presentation	talk-4		2022-11-21	Indianapolis, US		75th Annual Meeting of the American Physical Society (APS), Division of Fluid Dynamics (DFD)
Convoluted interfaces in laminar dispersing jets 	Poste

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,DNS and physics-informed surrogate models of s...,Oral Presentation,talk-1,NaN,2023-11-20,"Washington D.C., USA",NaN,76th Annual Meeting of the American Physical S...
1,Direct Numerical Simulation of an Alternative ...,Oral Presentation,talk-2,NaN,2023-11-07,"Orlando, USA",NaN,2023 AIChE Annual Meeting
2,Direct numerical simulations of the dispersion...,Oral Presentation,talk-3,NaN,2023-04-06,"Kobe, Japan",NaN,The 11th International Conference on Multiphas...
3,Direct numerical simulations of the dispersion...,Oral Presentation,talk-4,NaN,2022-11-21,"Indianapolis, US",NaN,75th Annual Meeting of the American Physical S...
4,Convoluted interfaces in laminar dispersing jets,Poster Presentation,poster-1,NaN,2022-11-21,"Indianapolis, US",NaN,75th Annual Meeting of the American Physical S...
5,On the dispersion dynamics of surfactant-laden...,Oral Presentation,talk-5,NaN,2022-11-15,"Phoenix, US",NaN,2022 AIChE Annual Meeting
6,Direct Numerical Simulation of the dispersion ...,Oral Presentation,talk-6,NaN,2022-06-07,"Oslo, Norway",NaN,8th European Congress on Computational Methods...
7,Towards Physics Informed Decision-making for C...,Poster Presentation,poster-2,NaN,2022-03-07,"London, U.K",NaN,Parliamentary and Scientific Committee's STEM ...
8,Direct numerical simulations of laminar drople...,Oral Presentation,talk-7,NaN,2021-11-22,"Phoenix, US",NaN,74th Annual Meeting of the American Physical S...
9,The journey of a drop in a static mixer,Poster Presentation,poster-3,NaN,2021-11-22,"Phoenix, US",NaN,74th Annual Meeting of the American Physical S...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2018-04-14-talk-9.md   2022-06-07-talk-6.md   2023-11-07-talk-2.md
2019-05-20-talk-8.md   2022-11-15-talk-5.md   2023-11-20-talk-1.md
2021-11-22-poster-3.md 2022-11-21-poster-1.md talkmap.ipynb
2021-11-22-talk-7.md   2022-11-21-talk-4.md
2022-03-07-poster-2.md 2023-04-06-talk-3.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
